In [110]:
import pandas as pd
import numpy as np
import plotly.express as px

from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge, RidgeClassifier, Lasso
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


import tensorflow as tf
import tensorflow_addons as tfa

In [6]:
data_base = pd.read_excel('./big_merge_V2_meteo_SAT.xlsx').drop('Unnamed: 0', axis=1)

In [13]:
data_base.loc[data_base['LFI']=='LFI4','DEGRAD_PPL'].value_counts()

 2    962
 1    806
 3    389
 4    146
 5     41
 6     34
-1     25
Name: DEGRAD_PPL, dtype: int64

In [14]:
data_base['DEGRAD_PPL'] = data_base['DEGRAD_PPL'].apply(lambda v : v if v!=-1 else 3)

In [15]:
data_base['TAUX_COUV_RAJ'].value_counts()

-1    2438
 2    2313
 3    2133
 4    1337
 5     629
 1     417
 6     345
Name: TAUX_COUV_RAJ, dtype: int64

PREPROCESSING _ Code base for models temporal predictions

Ici, features engineering (création de nouvelles features à partir de la liste connues):

In [16]:
# adding aridity index
data_base["AI"] = data_base['PRCP_GROWTH'] / data_base['TAVE_GROWTH']
# adding H/D index
data_base["H_D"] = data_base['HAUTEUR_ARBRE'] / data_base['DBH']

### Choix TARGET et Features :

Target :

In [60]:
TARGET = ['DEGRAD_PPL'] #exemple

Features :

In [61]:
# --- PAST --- 
# Attention : on peut, logiquement, inclure la TARGET... (connue dans le passé)
cat_strict = ['PRODREG', 'ESPECE_DOM', 'TYP_RAJ_PPL', 'DEG_FERMETURE', 'STR_PPL', 'RELIEF', 'ORIENTATION'] #exemple
cat_ord_miss = ['TAUX_COUV_RAJ', 'HT_VEG', 'DEGRAD_PPL'] #exemple 'TAILLE_PPL', 'MELANGE', 'QUAL_STATION', 'SURF_TROU_AER'
numerics = ['UNIT_VOL_BOIS_MANQUANT', 'H_D','AI','SDI', 'AGE_PPL','ALT', 'TIGES_VIV_H', 'SURF_TER_HA', 'FEUILL_PER', 'CONIF_PER', 'PERF_CROI', 'NDVI', 'EVI', 'NDMI', 'NDWI', 'DSWI'] #exemple

# --- FUTURE KNOWN ---
# Attention : logiquement les features, potentiellement connues dans le futur ci-dessous sont aussi répertoriées ci-dessus dans le passé
add_cat_IFN_stable = ['ESPECE_DOM', 'DEG_FERMETURE', 'STR_PPL'] # technos-substituts : 'DEG_FERMETURE', 'STR_PPL'
add_cat_ord_IFN_stable = [] # technos-substituts : 'DEG_FERMETURE', 'STR_PPL', 'SURF_TROU_AER' 'TAILLE_PPL', 'SURF_TROU_AER'
add_IFN_numerics_stable = ['H_D','AI'] # technos-substituts : , 'SDI'
# données potentiellement connues car stables par parcelles :
#['LAT', 'LON', 'ALT', 'PRODREG', 'HT_VEG', 'SLOPE25', 'ASPECT25', 'ORIENTATION', 'PERF_CROI', 'QUAL_STATION', 'UNIT_VEG_FINE',
# 'UNIT_VEG_GROS', 'PROCESS_SILVA', 'PERI_CHENAUX', 'PERI_COULEES','PERI_AVALANCH', 'PERI_CHUTES', 'ETAGE', 'ENDOMMAGEMENT','NB_DEGAT_ARBRE']
# + données extrapolées grâce à des technologies prometteuses (satellites ?)....

add_meteo_known = ['PRCP', 'TAVE_AVG',	'TAVE', 'TAVE_GROWTH', 'PRCP_S_S',	'PRCP_G_S']

add_SAT_known = ['NDVI', 'EVI', 'NDMI', 'NDWI', 'DSWI']

In [62]:
features_past = numerics + cat_strict + cat_ord_miss
features_future = add_cat_IFN_stable + add_cat_ord_IFN_stable + add_IFN_numerics_stable + add_meteo_known + add_SAT_known

In [63]:
data_LFI1 = data_base.loc[data_base['LFI']=='LFI1',:].sort_values('PARCELLE')
data_LFI2 = data_base.loc[data_base['LFI']=='LFI2',:].sort_values('PARCELLE')
data_LFI3 = data_base.loc[data_base['LFI']=='LFI3',:].sort_values('PARCELLE')
data_LFI4 = data_base.loc[data_base['LFI']=='LFI4',:].sort_values('PARCELLE')

In [64]:
future_feat_names = []
add_cat_strict_feat_names = []
add_cat_ord_feat_names = []
add_numerics_feat_names = []

for cat in features_future:
    lfi2_list = data_LFI2[cat].to_list()
    lfi3_list = data_LFI3[cat].to_list()
    lfi4_list = data_LFI4[cat].to_list()
    data_LFI1[cat + "_f"] = lfi2_list
    data_LFI2[cat + "_f"] = lfi2_list
    data_LFI3[cat + "_f"] = lfi2_list
    future_feat_names.append(cat + '_f')
    if cat in add_cat_ord_IFN_stable:
        add_cat_ord_feat_names.append(cat + '_f')
    elif cat in add_cat_IFN_stable:
        add_cat_strict_feat_names.append(cat + '_f')
    else:
        add_numerics_feat_names.append(cat + '_f')


In [65]:
data_red = pd.concat([data_LFI1, data_LFI2, data_LFI3], axis=0)[features_past + future_feat_names]
Y = pd.concat([data_LFI2[TARGET], data_LFI3[TARGET], data_LFI4[TARGET]], axis = 0)

In [69]:
data_red.head()

,UNIT_VOL_BOIS_MANQUANT,H_D,AI,SDI,AGE_PPL,ALT,TIGES_VIV_H,SURF_TER_HA,FEUILL_PER,CONIF_PER,...,TAVE_AVG_f,TAVE_f,TAVE_GROWTH_f,PRCP_S_S_f,PRCP_G_S_f,NDVI_f,EVI_f,NDMI_f,NDWI_f,DSWI_f
0,NaN,0.695652,NaN,571,85.0,715.918970,590.0,27.79,NaN,NaN,...,8.8859,9.026806,13.84776,117.278617,51.835496,0.4992,0.0160,0.2705,-0.4653,0.6865
5,NaN,0.647727,NaN,890,140.0,563.829759,400.0,53.38,NaN,NaN,...,8.9951,19.793490,13.83959,122.675652,55.827235,0.5552,0.0149,0.2017,-0.4864,0.6996
9,NaN,0.666667,NaN,489,80.0,564.885846,320.0,26.70,NaN,NaN,...,9.0637,11.671380,13.39843,128.067334,61.309991,0.5633,0.0153,0.1727,-0.5002,0.6852
14,NaN,NaN,NaN,0,2.0,563.551602,0.0,0.00,NaN,NaN,...,8.8805,10.459870,13.15530,130.759873,63.219754,0.6059,0.0197,0.2269,-0.5588,0.8384
19,NaN,0.676471,NaN,377,130.0,539.769096,150.0,23.32,NaN,NaN,...,8.8384,11.544840,13.16893,131.143585,62.702111,0.6012,0.0184,0.2029,-0.5486,0.7902


In [70]:
Y = Y.to_numpy().ravel()

Traitement des données catégorielles ordonnées en numériques (gestion des "-1" éventuels) :

In [72]:
feats_cat_ord = cat_ord_miss + add_cat_ord_feat_names
for cat in feats_cat_ord:
  data_red[cat] = data_red[cat].apply(lambda v : v if v!=-1 else np.nan)

LISTE DES FEATURES NUMERIQUES ET CATEGORIELLES EN VUE DU PREPROCESSING DE MODELE :

In [73]:
numerics_features = numerics + feats_cat_ord + add_numerics_feat_names
feats_cat_strict = cat_strict + add_cat_strict_feat_names

In [74]:
feats_cat_strict

['PRODREG',
 'ESPECE_DOM',
 'TYP_RAJ_PPL',
 'DEG_FERMETURE',
 'STR_PPL',
 'RELIEF',
 'ORIENTATION',
 'ESPECE_DOM_f',
 'DEG_FERMETURE_f',
 'STR_PPL_f']

In [75]:
numerics_transforms = Pipeline(
    [('imputer',KNNImputer()),
    ('encoder',StandardScaler())
])
categorials_transforms = Pipeline([
    ('imputer',KNNImputer()),
    ('encoder',OneHotEncoder(drop="first"))
])

preprocessor = ColumnTransformer(
    [("num", numerics_transforms, numerics_features),
     ("cat", categorials_transforms, feats_cat_strict)])

In [76]:
data_red['ORIENTATION'] = data_red['ORIENTATION'].map({'N':0,'NE':1,'E':2,'SE':3,'S':4,'SO':5,'O':6,'NO':7})
#data_red['ORIENTATION_f'] = data_red['ORIENTATION_f'].map({'N':0,'NE':1,'E':2,'SE':3,'S':4,'SO':5,'O':6,'NO':7})


In [77]:
X_train, X_test, y_train, y_test = train_test_split(data_red, Y, test_size=0.2, random_state=2, stratify=Y)

In [78]:
np.shape(X_train)

(5767, 42)

In [79]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [80]:
np.shape(X_train)

(5767, 102)

In [81]:
np.shape(X_test)

(1442, 102)

In [111]:
model =  RandomForestClassifier()

In [117]:
params = {
    'n_estimators' : [20, 40, 60],
    'max_depth' : [10, 12, 18],
    'min_samples_split' : [2, 4, 6] 
}

In [118]:
grid = GridSearchCV(model, param_grid=params, scoring='accuracy', verbose=2)

In [119]:
grid.fit(X_train, y_train)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV] END .max_depth=10, min_samples_split=2, n_estimators=20; total time=   0.2s
[CV] END .max_depth=10, min_samples_split=2, n_estimators=20; total time=   0.2s
[CV] END .max_depth=10, min_samples_split=2, n_estimators=20; total time=   0.2s
[CV] END .max_depth=10, min_samples_split=2, n_estimators=20; total time=   0.2s
[CV] END .max_depth=10, min_samples_split=2, n_estimators=20; total time=   0.2s
[CV] END .max_depth=10, min_samples_split=2, n_estimators=40; total time=   0.4s
[CV] END .max_depth=10, min_samples_split=2, n_estimators=40; total time=   0.4s
[CV] END .max_depth=10, min_samples_split=2, n_estimators=40; total time=   0.4s
[CV] END .max_depth=10, min_samples_split=2, n_estimators=40; total time=   0.4s
[CV] END .max_depth=10, min_samples_split=2, n_estimators=40; total time=   0.4s
[CV] END .max_depth=10, min_samples_split=2, n_estimators=60; total time=   0.7s
[CV] END .max_depth=10, min_samples_split=2, n_

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'max_depth': [10, 12, 18],
                         'min_samples_split': [2, 4, 6],
                         'n_estimators': [20, 40, 60]},
             scoring='accuracy', verbose=2)

In [120]:
grid.best_estimator_

RandomForestClassifier(max_depth=12, n_estimators=60)

In [121]:
train_scores = cross_val_score(grid.best_estimator_, X_train, y_train, cv=5)
test_scores = cross_val_score(grid.best_estimator_, X_test, y_test, cv=5)
print(f'Train score mean : {np.mean(train_scores)}')
print(f'Train score std : {np.std(train_scores)}')
print(f'Test score mean : {np.mean(test_scores)}')
print(f'Test score std : {np.std(test_scores)}')

Train score mean : 0.5129201044370725
Train score std : 0.014280228961167292
Test score mean : 0.461868031526336
Test score std : 0.015361416782259364


In [108]:
list_features_in = []
for feat in numerics_features:
  list_features_in.append(feat)
for cat in feats_cat_strict:
  nb_lab = len(data_red[cat].unique())-1
  for i in range(nb_lab):
    list_features_in.append(f'{cat}_{i}')

In [109]:
df_coef_inter = pd.DataFrame(grid.best_estimator_.coef_)

AttributeError: 'DecisionTreeClassifier' object has no attribute 'coef_'

In [43]:
len(abs(df_coef_inter).sum())

102

In [44]:
df_coef = pd.DataFrame(abs(df_coef_inter).sum(), columns=['Coeff'])

In [45]:
df_coef

,Coeff
0,0.114190
1,0.077257
2,0.041447
3,1.033770
4,0.145568
...,...
97,0.287888
98,0.233795
99,0.290973
100,0.317581


In [46]:
len(list_features_in)

102

In [47]:
df_coef['Features'] = list_features_in

In [48]:
df_coef = df_coef.set_index('Features')

In [49]:
fig = px.bar(df_coef['Coeff'], title=f"Features importance for target : {TARGET} with Lasso Linear Regression")
fig.show()

-----------------

In [122]:
y_train

array([1, 2, 1, ..., 2, 3, 1], dtype=int64)

In [123]:
processor_y = OneHotEncoder(sparse=False)
y_train = processor_y.fit_transform(y_train.reshape(-1, 1))
y_test = processor_y.transform(y_test.reshape(-1, 1))

In [124]:
train_batch = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(buffer_size=len(X_train)).batch(batch_size=32)
test_batch = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size=32)

In [125]:
for x, y in train_batch.take(1):
    print(x)
    print(y)

tf.Tensor(
[[ 0.81232183 -0.42325376  2.65827134 ...  0.          0.
   0.        ]
 [ 1.02588955 -0.76580331 -0.17188842 ...  0.          0.
   0.        ]
 [-0.31680143 -1.44645392 -0.0788555  ...  0.          1.
   0.        ]
 ...
 [-0.47837006 -0.0479766   0.59929746 ...  0.          1.
   0.        ]
 [-0.51551227 -0.00513676 -1.07505085 ...  1.          0.
   0.        ]
 [-0.21837457  1.84769052  0.68961715 ...  1.          0.
   0.        ]], shape=(32, 102), dtype=float64)
tf.Tensor(
[[0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0

In [129]:
model = tf.keras.models.Sequential([
        tf.keras.Input(shape=(102,)),
        tf.keras.layers.Dense(64,"relu"),
        tf.keras.layers.Dense(32,"relu"),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(16,"relu"),
        tf.keras.layers.Dense(6,"softmax")
    ])

In [130]:
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=tf.keras.metrics.CategoricalAccuracy())

In [131]:
model.fit(train_batch, epochs=20, validation_data=test_batch)

Epoch 1/20
181/181 [==============================] - 3s 10ms/step - loss: 1.3210 - categorical_accuracy: 0.4014 - val_loss: 1.2262 - val_categorical_accuracy: 0.4272
Epoch 2/20
181/181 [==============================] - 2s 9ms/step - loss: 1.1829 - categorical_accuracy: 0.4644 - val_loss: 1.1764 - val_categorical_accuracy: 0.4653
Epoch 3/20
181/181 [==============================] - 2s 9ms/step - loss: 1.1395 - categorical_accuracy: 0.4961 - val_loss: 1.1598 - val_categorical_accuracy: 0.4834
Epoch 4/20
181/181 [==============================] - 2s 9ms/step - loss: 1.1104 - categorical_accuracy: 0.5138 - val_loss: 1.1545 - val_categorical_accuracy: 0.4827
Epoch 5/20
181/181 [==============================] - 2s 9ms/step - loss: 1.0937 - categorical_accuracy: 0.5330 - val_loss: 1.1468 - val_categorical_accuracy: 0.4951
Epoch 6/20
181/181 [==============================] - 2s 9ms/step - loss: 1.0787 - categorical_accuracy: 0.5351 - val_loss: 1.1493 - val_categorical_accuracy: 0.4799
Epo

In [134]:
coeff_mean = []
for i in range(102):
    coeff_mean.append(np.mean(model.layers[0].trainable_variables[0][i]))

In [135]:
df_coef = pd.DataFrame(coeff_mean, columns=['Coeff'], index= list_features_in)

In [136]:
fig = px.bar(df_coef['Coeff'], title=f"Features importance for target : {TARGET} with Lasso Linear Regression")
fig.show()